In [87]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.rozee.pk/job/jsearch/q/Software%20Engineer")
time.sleep(3)

job_list = []
job_count = 0
min_jobs = 30

while job_count < min_jobs:
    jobs = driver.find_elements(By.CSS_SELECTOR, "div.job")
    job_index = 0

    while job_index < len(jobs) and job_count < min_jobs:
        try:
            job = jobs[job_index]
            title = job.find_element(By.CSS_SELECTOR, "h3.s-18").text
            company_location = job.find_element(By.CSS_SELECTOR, "div.cname").text.split(', ')
            company = company_location[0]
            location = company_location[1] if len(company_location) > 1 else "Not specified"
            skills = job.find_element(By.CSS_SELECTOR, "div.job-dtl.clearfix").text.split('\n')

            job_link_element = job.find_element(By.XPATH, ".//a")
            job_link = job_link_element.get_attribute("href") if job_link_element else None

            if job_link:
                driver.get(job_link)
                time.sleep(3)

                description = driver.find_element(By.CSS_SELECTOR, "div.jblk.ul18").text
                job_type = driver.find_element(By.XPATH, "//a[contains(@href, '-jobs-in-pakistan')]").text

                job_list.append([title, company, location, skills, job_type, description])
                job_count += 1

                driver.back()
                time.sleep(3)
                jobs = driver.find_elements(By.CSS_SELECTOR, "div.job")

            job_index += 1

        except Exception as e:
            print(f"Error processing job: {e}")
            job_index += 1

    if job_count < min_jobs:
        try:
            next_button = driver.find_element(By.LINK_TEXT, "Next")
            next_button.click()
            time.sleep(3)
        except:
            break
driver.quit()

job_data = pd.DataFrame(job_list, columns=["Job Title", "Company Name", "Location", "Required Skills", "Job Type", "Job Description"])
job_data.to_csv("rozee_jobs.csv", index=False)


Error processing job: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h3.s-18"}
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x004C74A3+25091]
	(No symbol) [0x0044DC04]
	(No symbol) [0x0032B373]
	(No symbol) [0x0036F4DC]
	(No symbol) [0x0036F65B]
	(No symbol) [0x00364F21]
	(No symbol) [0x00391F54]
	(No symbol) [0x00364E44]
	(No symbol) [0x003921A4]
	(No symbol) [0x003AB49E]
	(No symbol) [0x00391CA6]
	(No symbol) [0x003631D5]
	(No symbol) [0x0036435D]
	GetHandleVerifier [0x007C07C3+3142947]
	GetHandleVerifier [0x007D1A2B+3213195]
	GetHandleVerifier [0x007CC412+3191154]
	GetHandleVerifier [0x00568720+685184]
	(No symbol) [0x00456E1D]
	(No symbol) [0x00453E18]
	(No symbol) [0x00453FB6]
	(No symbol) [0x004466F0]
	BaseThreadInitThunk [0x76327BA9+25]
	RtlInitializeExceptionChain [0

In [88]:
job_data

,Job Title,Company Name,Location,Required Skills,Job Type,Job Description
0,Software Engineer,Croem Inc,Islamabad,[DotNet],Full Time,Croem Inc is seeking a talented Software Engin...
1,Software Engineer,St. Michael Llc,Karachi,"[JavaScript, Java, CPLUSPLUS]",Full Time,St. Michael LLC is seeking a skilled Software ...
2,Software Engineer,Wayne Enterprises,All Cities,[JavaScript],Full Time,Wayne Enterprises is seeking a skilled Softwar...
3,Software Engineer,Xoftex,Lahore,"[Microsoft .NET, .Net, AngularJS]",Full Time,We are looking for a Software Engineer to join...
4,Software Engineer,Softo Tech,Islamabad,"[Knowledge of Digital Marketing, Knowledge of ...",Contract,Knowledge of Digital marketing \nAbilities of ...
5,Software Engineer,Digi Soft,Karachi,"[ASP.net, MVC, RESTful APIs, AngularJS]",Full Time,"Good in Anuglar, Net core, and Microsoft SQL S..."
6,Software Engineer - SQL Database,The Shams Group,Multiple Cities,"[JavaScript, HTML, Data Management, .NET Frame...",Full Time,We are looking for a qualified Database develo...
7,Software Engineer,Universal Technology Systems and Associates LLC,All Cities,"[Backend Development, Jira Configuration, HTML...",Full Time,"Create, modify, and test code and scripts for ..."
8,Software Engineer,MATZ Solutions Pvt Ltd,Karachi,"[JavaScript, Automation Languages Command, Exe...",Full Time,We are looking for talented and motivated Soft...
9,Senior Software Engineer,Pakistan Single Window,Karachi,"[ASP.net, JavaScript, jQuery, Software Analysi...",Contract,Main Responsibilities:\nDesign and develop mic...


In [89]:
from collections import Counter

top_titles = job_data["Job Title"].value_counts()
print("Most Common Job Titles:\n", top_titles)

all_skills = [skill for skills in job_data["Required Skills"] for skill in skills]
skill_counts = Counter(all_skills)
top_skills = skill_counts.most_common()
print("\nMost Frequently Required Skills:\n", top_skills)

Most Common Job Titles:
 Job Title
Software Engineer                      8
Software Quality Assurance Engineer    7
Mechanical Engineer                    2
Software Engineer - SQL Database       1
Senior Software Engineer               1
Principal Software Engineer            1
Software Engineer (Node.Js)            1
Software Support Engineer              1
Full Stack Software Engineer           1
Senior Frontend Software Engineer      1
QA Automation Engineer                 1
Electrical Engineer                    1
Civil Engineer                         1
Field Engineer                         1
Design Engineer                        1
IT Support Engineer                    1
Name: count, dtype: int64

Most Frequently Required Skills:
 [('JavaScript', 6), ('Software Quality Review', 5), ('ASP.net', 4), ('HTML', 4), ('Performance Testing', 4), ('Communication Skills', 4), ('.Net', 3), ('Software Development', 3), ('CCHANNEL', 3), ('Software Life Cycle Testing', 3), ('AutoCAD Desig

In [93]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.youtube.com/results?search_query=Deepseek&sp=EgIIBQ%253D%253D")
time.sleep(3)

for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(3)

videos = driver.find_elements(By.XPATH, "//ytd-video-renderer")
video_data = []

for video in videos[:25]:
    try:
        title = video.find_element(By.ID, "video-title").text
        # channel = video.find_element(By.CSS_SELECTOR, "div#text-container.style-scope.ytd-channel-name").text
        views = video.find_element(By.XPATH, ".//span[contains(text(), 'views')]").text
        upload_date = video.find_element(By.XPATH, ".//div[@id='metadata-line']/span[2]").text
        
        video_url = video.find_element(By.ID, "video-title").get_attribute("href")
        driver.execute_script("window.open(arguments[0], '_blank');", video_url)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3)

        try:
            duration = driver.find_element(By.CLASS_NAME, "ytp-time-wrapper").text
            duration = duration.split('/ ')[-1] if '/ ' in duration else duration
        except:
            duration = "N/A"
        
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div#snippet.style-scope.ytd-text-inline-expander").text
        except:
            description = "No description"
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        video_data.append([title, views, upload_date, duration, description])
    except:
        continue
driver.quit()

video_df = pd.DataFrame(video_data, columns=["Title", "Views", "Upload Date", "Duration", "Description"])
video_df.to_csv("youtube_videos.csv", index=False)


In [94]:
video_df

,Title,Views,Upload Date,Duration,Description
0,How A Chinese Villager Shook Silicon Valley [D...,406K views,5 days ago,0:10,Go to https://ground.news/coldfusion to stay f...
1,OpenAI's nightmare: Deepseek R1 on a Raspberry Pi,2M views,2 weeks ago,0:34,"DeepSeek R1 runs on a Pi 5, but don't believe ..."
2,What’s Really Happening with DeepSeek,4.3M views,2 weeks ago,,
3,What is DeepSeek? How To Use It? | ChatGPT Killer,116K views,2 weeks ago,,
4,What is DeepSeek? AI Model Basics Explained,127K views,10 days ago,10:21,Want to learn more about how to choose the rig...
5,How to Earn with DeepSeek AI | Create a Tool W...,32K views,5 days ago,20:23,Get your Hosting : https://dskills.online/\n📢 ...
6,Deepseek | China's New AI Model Destroys Ameri...,8.8M views,12 days ago,,🤖 Join AI Chatbots Course: https://academy.dhr...
7,The Man Behind DeepSeek (Liang Wenfeng),424K views,2 weeks ago,18:02,Discover China’s Best – Best Prices on Trip.co...
8,China's BYD Challenges Tesla With Plans to Add...,45K views,5 days ago,2:18,BYD Co. will enable advanced driver-assistance...
9,the ONLY way to run Deepseek...,853K views,2 weeks ago,11:58,☕Because everything in I.T. requires coffee: h...


In [95]:
video_df[['Views_Num', 'Views_Suffix']] = video_df['Views'].str.extract(r'([\d\.]+)([KMB]?)')
suffix_mapper = {'': 1,'K': 1000, 'M': 1000000, 'B': 1000000000}
video_df['Views_Total'] = video_df['Views_Num'].astype(float) * video_df['Views_Suffix'].map(suffix_mapper)

most_viewed_video = video_df.iloc[video_df['Views_Total'].idxmax()]
print("Most viewed video:", most_viewed_video)

avg_duration = video_df['Duration'].str.extract('(\d+):(\d+)').astype(float).fillna(0).apply(lambda x: x[0] * 60 + x[1], axis=1).mean()
print("Average video duration (seconds):", avg_duration)

Most viewed video: Title           Deepseek | China's New AI Model Destroys Ameri...
Views                                                  8.8M views
Upload Date                                           12 days ago
Duration                                                         
Description     🤖 Join AI Chatbots Course: https://academy.dhr...
Views_Num                                                     8.8
Views_Suffix                                                    M
Views_Total                                             8800000.0
Name: 6, dtype: object
Average video duration (seconds): 496.6


<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Safwan\AppData\Local\Temp\ipykernel_3932\2333103495.py:8: SyntaxWarning: invalid escape sequence '\d'
  avg_duration = video_df['Duration'].str.extract('(\d+):(\d+)').astype(float).fillna(0).apply(lambda x: x[0] * 60 + x[1], axis=1).mean()


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website_link = "https://www.lamaretail.com/"
driver.get(website_link)
time.sleep(3)
lamaretail_data = []

women_link = website_link + "collections/woman-spring-summer-25"
driver.get(women_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "a.grid-product__link")
for product in products[:10]:
    product_info = product.find_element(By.CLASS_NAME, "grid-product__meta").text
    product_name, product_price = product_info.split('\n')
    lamaretail_data.append([product_name, product_price, "Women"])

driver.back()
time.sleep(3)
men_link = website_link + "collections/man-spring-summer-25"
driver.get(men_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "a.grid-product__link")
for product in products[:10]:
    product_info = product.find_element(By.CLASS_NAME, "grid-product__meta").text
    product_name, product_price = product_info.split('\n')
    lamaretail_data.append([product_name, product_price, "Men"])

shoes_link = website_link + "collections/shoes-spring-summer-25"
driver.get(shoes_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "a.grid-product__link")
for product in products[:10]:
    product_info = product.find_element(By.CLASS_NAME, "grid-product__meta").text
    product_name, product_price = product_info.split('\n')
    lamaretail_data.append([product_name, product_price, "Shoes"])

women_bag_link = website_link + "collections/woman-bags"
driver.get(women_bag_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "a.grid-product__link")
for product in products[:10]:
    product_info = product.find_element(By.CLASS_NAME, "grid-product__meta").text
    product_name, product_price = product_info.split('\n')
    lamaretail_data.append([product_name, product_price, "Women Bags"])

men_socks_link = website_link + "collections/man-socks"
driver.get(men_socks_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "a.grid-product__link")
for product in products[:10]:
    product_info = product.find_element(By.CLASS_NAME, "grid-product__meta").text
    product_name, product_price = product_info.split('\n')
    lamaretail_data.append([product_name, product_price, "Men Socks"])

fragrance_link = website_link + "collections/fragrance"
driver.get(fragrance_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "a.grid-product__link")
for product in products[:10]:
    product_info = product.find_element(By.CLASS_NAME, "grid-product__meta").text
    product_name, product_price = product_info.split('\n')
    lamaretail_data.append([product_name, product_price, "Fragrance"])

driver.quit()
lamaretail_df = pd.DataFrame(lamaretail_data, columns=["Name", "Price", "Category"])
lamaretail_df.to_csv("lamaretail_products.csv", index=False)


In [91]:
lamaretail_df

,Name,Price,Category
0,CROPPED BUTTON DOWN,"Rs.7,450.00",Women
1,LINEN SAINT TROPEZ BLOUSE,"Rs.6,450.00",Women
2,LINEN CULOTTES,"Rs.6,950.00",Women
3,JUST ANOTHER BRAND OVERSIZED TEE,"Rs.3,450.00",Women
4,COTTON BODYSUIT,"Rs.3,950.00",Women
5,BASIC SILK SHIRT,"Rs.6,450.00",Women
6,STRIPED MUSLIN SHIRT,"Rs.6,950.00",Women
7,JUST PEACH TUNIC,"Rs.5,950.00",Women
8,STRIPED TANK TOP,"Rs.3,450.00",Women
9,ORGANIC MUSLIN SHIRT,"Rs.6,950.00",Women


In [92]:
lamaretail_df['Price'] = lamaretail_df['Price'].str.replace('Rs.', '', regex=False)
lamaretail_df['Price'] = lamaretail_df['Price'].str.replace(',', '', regex=False)
lamaretail_df['Price'] = lamaretail_df['Price'].str.split('.').str[0]
lamaretail_df['Price'] = lamaretail_df['Price'].astype(int)

average_prices = lamaretail_df.groupby('Category')['Price'].mean().reset_index()
print(average_prices)

     Category    Price
0   Fragrance   7450.0
1         Men   8750.0
2   Men Socks    570.0
3       Shoes   7150.0
4       Women   5800.0
5  Women Bags  10400.0
